In [1]:
using DynamicPolynomials,LinearAlgebra

##  Input
- Matrix $H$ and vector $d$ so that $\mathcal{X} = \lbrace x \in \mathbb{R}^n | Ax=c \rbrace$
- Signomial defined by Matrix $A^{m \times n}$ and coeffiecients $c \in \mathbb{R}^m$
##  Example

In [2]:
c = [1,-1,-1]
A=[3//2 0;0 1;0 -1]

3×2 Matrix{Rational{Int64}}:
 3//2   0//1
 0//1   1//1
 0//1  -1//1

In [3]:
H=[1 0;-1 0;0 1;0 -1]
d=[1,-1,1,1]

4-element Vector{Int64}:
  1
 -1
  1
  1

At first we need to calculate the leftinverse of A

In [4]:
function rpinv(A) ## rational leftinverse
    rA = Rational.(A)
    B = (transpose(rA)*rA)\transpose(rA)
    return B
end

rpinv (generic function with 1 method)

In [5]:
function getSupport(B,d)
    Supp=[]
    nvars=size(B)[2]
    # print(nvars)
    for (i,row) in enumerate(eachrow(B))
        P=[]
        N=[]
        denom=lcm(denominator.(row))
        for (j,ele) in enumerate(row)
            if ele >= 0
                push!(P,(Int(ele*denom),j))
            else
                push!(N,(Int(-ele*denom),j))
            end
        end




        pdegree=sum(Int[x[1] for x in P])
        ndegree=sum(Int[x[1] for x in N])
        if pdegree>ndegree
            push!(N,(pdegree-ndegree,nvars+1))
        elseif pdegree<ndegree
            push!(P,(ndegree-pdegree,nvars+1))   
        end
        push!(Supp,(P,N,denom*d[i]))
    end
    return Supp
end

getSupport (generic function with 1 method)

In [6]:
function getSASet(Supp)
    F=[]
    for ele in Supp
        s=ele
        P=s[1]
        N=s[2]
        dᵢ=s[3]
        deg = size(P)[1]+size(N)[1]
        @polyvar x[1:deg]
        f=x[1]*0
        ## Add positive part
        p=x[1]*0+1
        for ele in P
            p=p*x[ele[2]]^ele[1]
        end
        f=f-p
        ##Add negative part
        p=x[1]*0+ℯ^dᵢ
        for ele in N
            p=p*x[ele[2]]^ele[1]
        end
        f=f+p
        push!(F,f)
    end
    return F
end

getSASet (generic function with 1 method)

In [7]:
function SASbyMatrix(A,H,c,d)
    ##ToDo Check Dimensions
    L=rpinv(A)
    B=H*L
    Supp=getSupport(B,d)
    return getSASet(Supp)
end

SASbyMatrix (generic function with 1 method)

In [8]:
SASbyMatrix(A,H,c,d)

4-element Vector{Any}:
 -x₁² + 20.085536923187668x₄²
 0.049787068367863944x₁² - x₄²
 -x₂ + 7.38905609893065x₃
 7.38905609893065x₂ - x₃